1\. Write a function that converts number representation, bin<->dec<->hex. (Clearly using the corresponding python built-in functions is not fair..)

In [1]:
def DecToBin(n):
    remainders=[]
    while int(n)>0:
        remainders.append(str(n%2))
        n=n//2
    return "".join(remainders[::-1])
    
def BinToDec(s):
    d=0
    for i,c in enumerate(s[::-1]):
        d+=int(c)*2**i
    return d

DecToBin(42), BinToDec(DecToBin(42)), BinToDec('0101')

('101010', 42, 5)

2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

Take a look [here](https://www.h-schmidt.net/FloatConverter/IEEE754.html) to check

In [2]:
def WordToFloat(s):
    sgn=int(s[0])
    exp=BinToDec(s[1:9])
    man=BinToDec(s[9:32])
    d_man=man/(2**23)
    print('sgn',sgn)
    print('exp',exp)
    print('d_man', d_man)
    return ((-1)**sgn)*(2**(exp-127))*(1+d_man)

WordToFloat('01000010001010000000000000000000')

sgn 0
exp 132
d_man 0.3125


42.0

3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [3]:
underflow=1.0
attempts=1075

for i in range(attempts):
    underflow=underflow/2
    if underflow==0:
        print(i+1, underflow)
        break

1075 0.0


In [4]:
import math
overflow=1.0
for i in range(attempts):
    overflow=overflow*2
    if overflow==math.inf:
        print(i+1, overflow)
        break

1024 inf


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [5]:
a=1.0
i=1
while True:
    if a==a+10**(-i):
        print('So... the limit is 10^-'+str(i) )
        break
    i+=1

So... the limit is 10^-16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [6]:
def roots(a,b,c):
    delta=b*b-4*a*c
    x1=(-b+delta**(0.5))/2*a
    x2=(-b-delta**(0.5))/2*a
    return x1,x2

def roots2(a,b,c):
    num=4*a*c
    delta=b*b-4*a*c
    x1=num/(2*a*(-b-delta**(0.5)))
    x2=num/(2*a*(-b+delta**(0.5)))
    return x1,x2

roots(0.001,1000,0.001), roots2(0.001,1000,0.001)

((-9.999894245993345e-13, -0.999999999999),
 (-1.0000000000009999e-06, -1000010.5755125057))

6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [7]:
def f(x):
    return x*(x-1)

def der_def(delta, f, x):
    return (f(x+delta)-f(x))/delta

deltas=[10**(-2*i) for i in range(1,8)]
for delta in deltas:
    d=der_def(delta,f, 1)
    print('Delta: {delta:.2e}\t Der: {der:.10f}\tErr: {err:.2e}'.format(delta=delta, der=d, err=(abs(d-1)/1*100)) )

Delta: 1.00e-02	 Der: 1.0100000000	Err: 1.00e+00
Delta: 1.00e-04	 Der: 1.0001000000	Err: 1.00e-02
Delta: 1.00e-06	 Der: 1.0000009999	Err: 1.00e-04
Delta: 1.00e-08	 Der: 1.0000000039	Err: 3.92e-07
Delta: 1.00e-10	 Der: 1.0000000828	Err: 8.28e-06
Delta: 1.00e-12	 Der: 1.0000889006	Err: 8.89e-03
Delta: 1.00e-14	 Der: 0.9992007222	Err: 7.99e-02


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [8]:
import math
import time
def func(x):
    return (1-x**2)**(0.5)

def timer(func):
    def func_wrapper(*args, **kwargs):
        t_s=time.time()
        ret=func(*args, **kwargs)
        t_total=time.time()-t_s
        print('Total time: %.2e sec'%t_total)
        return ret
    return func_wrapper

@timer
def integrate(start, stop, N, func):
    dx=(stop-start)/N
    integral=0
    for i in range(N):
        x=start+dx*i
        integral+=func(x)*dx
    return integral

def perc_error(i,N):
    pi_half=integrate(-1,1,N, func)
    perc=abs(pi_half- math.pi/2)/(math.pi/2)*100
    print('Res: ',pi_half, '\tErr: %.1e'%perc,'%','\nTru: ', math.pi/2, '\tSub: 10^',i, sep='')

subdiv=[10**i for i in range(2,9)]
subdiv.append(2*100000000)
for i,n in enumerate(subdiv):
    perc_error(i+2,n)

Total time: 4.51e-05 sec
Res: 1.5691342555492505	Err: 1.1e-01%
Tru: 1.5707963267948966	Sub: 10^2
Total time: 3.04e-04 sec
Res: 1.570743738501071	Err: 3.3e-03%
Tru: 1.5707963267948966	Sub: 10^3
Total time: 2.93e-03 sec
Res: 1.570794663715291	Err: 1.1e-04%
Tru: 1.5707963267948966	Sub: 10^4
Total time: 6.27e-02 sec
Res: 1.5707962742034223	Err: 3.3e-06%
Tru: 1.5707963267948966	Sub: 10^5
Total time: 2.93e-01 sec
Res: 1.5707963251317274	Err: 1.1e-07%
Tru: 1.5707963267948966	Sub: 10^6
Total time: 2.92e+00 sec
Res: 1.5707963267423612	Err: 3.3e-09%
Tru: 1.5707963267948966	Sub: 10^7
Total time: 2.90e+01 sec
Res: 1.570796326793284	Err: 1.0e-10%
Tru: 1.5707963267948966	Sub: 10^8
Total time: 6.41e+01 sec
Res: 1.5707963267944416	Err: 2.9e-11%
Tru: 1.5707963267948966	Sub: 10^9
